In [1]:
#settings 
from settings.settings import data_settings 
from settings.settings import model_settings

#imports
import pandas as pd 
# import numpy as np 
# import tensorflow as tf
# import keras
# from keras import layers
# import pathlib
# import datetime
# from model.blocks.keras_tcn import TCN


In [2]:
# Loading data and making it ready

train_df = pd.read_csv(data_settings.data_train_path)

test_df = pd.read_csv(data_settings.data_test_path)

test_buy = test_df.pop(test_df.buy_4.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.sell_4.name)

train_buy = train_df.pop(train_df.buy_4.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.sell_4.name)

train_date_time = train_df.pop(train_df.DateTime.name)
test_date_time = test_df.pop(test_df.DateTime.name)


In [ ]:
# Normalizing with z-score https://developers.google.com/machine-learning/data-prep/transform/normalization

# train_df.mean().to_csv(f"{data_settings.base_path}/data/csv/train_mean.csv")
# train_df.std().to_csv(f"{data_settings.base_path}/data/csv/train_std.csv")
# train_df = (train_df - train_df.mean())/train_df.std()
# test_df = (test_df - train_df.mean())/train_df.std()

df = (train_df - train_df.mean())/train_df.std()
df["DateTime"] = train_date_time


# Corrilation of buy and sell and do nothing with other rows of dataframes
# pd.options.display.max_rows =1000

# train_df.corr()["buy_4"]
# train_df.corr()["sell_4"]
# train_df.corr()["do_nothing"]

# import matplotlib.pyplot as plt

# for column in df.columns:
    
#     hist = df[column].hist(bins=100)
#     plt.savefig(f"./tmp/{column}_normalized.png")
#     hist.clear()


In [4]:
# y_train and y_test
# test_labels = (
#                 test_buy.to_numpy(),
#                 test_nothing.to_numpy(),
#                 test_sell.to_numpy()
#                )
# test_labels = np.array(test_labels)
# test_labels = test_labels.transpose()
# test_labels = test_labels.astype("float32")

# train_labels = (
#                 train_buy.to_numpy(),
#                 train_nothing.to_numpy(),
#                 train_sell.to_numpy()
#                )
# train_labels = np.array(train_labels)
# train_labels = train_labels.transpose()
# train_labels = train_labels.astype("float32")

y_train = pd.DataFrame([train_buy,train_nothing,train_sell]).transpose()
y_test = pd.DataFrame([test_buy,test_nothing,test_sell]).transpose()

# train_inputs = (train_df - train_df.mean())/train_df.std()
# test_inputs = (test_df - train_df.mean())/train_df.std()

X_train = train_df
X_test = test_df

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(278704, 97) (278704, 3) (49184, 97) (49184, 3)


In [ ]:
# Creating generators

# window_size = data_settings.window_size
# batch_size : int = 256
# num_features : int = 97
# num_classes : int = 3

# def x_train_generator():
#     for i in range(window_size,len(train_df)+1):
#         window=train_df[i-window_size:i]
#         yield window.to_numpy()

# def x_test_generator():
#     for i in range(window_size,len(test_df)+1):
#         window=test_df[i-window_size:i]
#         yield window.to_numpy()    
        
# def y_train_generator():
#     for i in range(window_size,train_labels.shape[0]+1):
#         yield train_labels[i-1]

# def y_test_generator():
#     for i in range(window_size,test_labels.shape[0]+1):
#         yield test_labels[i-1]
        
# x_train_ds = tf.data.Dataset.from_generator(x_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)

# x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)
# y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)
# y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)

# train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
# test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

In [ ]:
# def compile_and_fit(model:keras.Model,modelname,train_ds=train_ds,test_ds=test_ds,modelcheckpoint:bool = False,epochs= 10):
#     import os

#     #check if path is available
#     path = f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit'
#     pathlib.Path(path).mkdir(parents=True,exist_ok=True)

#     #tensorboard
#     log_dir =f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("(%Y-%m-%d)-(%H-%M-%S)")}'
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     #modelcheckpoint
#     base_path = f"./model/models/{modelname}/"
#     checkpoint_path = base_path + "tmp/cp-{epoch:04d}.ckpt"
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#         filepath=checkpoint_path,
#         monitor='val_loss',
#         verbose=1,
#         save_weights_only=True,
#         save_best_only=False
#     )

#     model.save_weights(checkpoint_path.format(epoch=0))


#     model.compile(loss='categorical_crossentropy',
#                     optimizer='adam',
#                     metrics=["categorical_accuracy","categorical_crossentropy"],
#                 )

#     if modelcheckpoint:
#         history = model.fit(
#                             train_ds,
#                             validation_data= test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback,model_checkpoint_callback],
#                             use_multiprocessing=True,
#                             )
#     else:
#         history = model.fit(    
#                             train_ds,
#                             validation_data=test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback],
#                             use_multiprocessing=True,
#                            )
    
#     return history

In [ ]:
#TF Model
# input_layer = layers.Input(shape=(window_size,num_features))
# x = TCN(
#     nb_filters=128,
#     # kernel_size=3,
#     # nb_stacks=1,
#     # padding="causal",
#     # activation="swish",
#     use_batch_norm=True,
#     # return_sequences=True
#     )(input_layer)

# # x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )

# input_layer = layers.Input(shape=(window_size,num_features))

# x = layers.Dense(1024,activation="relu")(input_layer)
# x = layers.Dense(512,activation="relu")(x)
# x = layers.Dense(256,activation="relu")(x)
# x = layers.Dense(128,activation="relu")(x)
# x = layers.Dense(64,activation="relu")(x)
# x = layers.Dense(32,activation="relu")(x)
# x = layers.Dense(16,activation="relu")(x)
# x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )
# model.summary()
# compile_and_fit(
#     model=model,
#     modelname="simple_dense_first_try",
#     train_ds=train_ds,
#     test_ds=test_ds,
#     modelcheckpoint=True
# )

In [5]:
from xgboost import XGBRFClassifier
from sklearn.metrics import log_loss , precision_score

model = XGBRFClassifier(
    learning_rate = 0.001,
    n_estimators = 100_000,
    verbosity = 3,
    tree_method = "gpu_hist",
    random_state = 23,  
    gpu_id = 0,
    eval_metric = log_loss
)

model = XGBRFClassifier(base_score=0.5,
                        booster='gbtree',
                        callbacks=None,
                        colsample_bylevel=1,
                        colsample_bytree=1,
                        early_stopping_rounds=None,
                        enable_categorical=False,
                        gamma=0,
                        gpu_id=0,
                        grow_policy='depthwise',
                        importance_type=None,
                        interaction_constraints='',
                        learning_rate=0.001,
                        max_bin=256,
                        max_cat_to_onehot=4,
                        max_delta_step=0,
                        max_depth=6,
                        max_leaves=0,
                        min_child_weight=1,
                        missing="nan",
                        monotone_constraints='()',
                        n_estimators=100000,
                        n_jobs=0,
                        num_parallel_tree=100000,
                        objective='binary:logistic',
                        predictor='auto',
                        random_state=23,
                        reg_alpha=0,
                        sampling_method='uniform')

In [6]:
# eval_set = [(X_test, y_test)]

model.fit(
            X_train,
            y_train,
            # eval_set = eval_set,
            verbose = 3
        )


[10:33:13] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_gpu_hist.cu:817: [GPU Hist]: Configure
[10:33:14] ======== Monitor (0): SketchContainer ========
[10:33:14] MakeCuts: 0.00086s, 1 calls @ 860us

[10:33:14] Prune: 0.000589s, 1 calls @ 589us

[10:33:14] ScanInput: 0.003684s, 1 calls @ 3684us

[10:33:14] Unique: 0.00036s, 1 calls @ 360us

[13:33:52] ======== Monitor (0): Learner ========
[13:33:52] Configure: 4e-06s, 1 calls @ 4us

[13:33:52] EvalOneIter: 0.000568s, 1 calls @ 568us

[13:33:52] GetGradient: 0.004797s, 1 calls @ 4797us

[13:33:52] PredictRaw: 0.495163s, 1 calls @ 495163us

[13:33:52] UpdateOneIter: 0s, 1 calls @ 0us

[13:33:52] ======== Monitor (0): GBTree ========
[13:33:52] BoostNewTrees: 10809.2s, 1 calls @ 10809223697us

[13:33:52] CommitModel: 0.008098s, 1 calls @ 8098us

[13:33:52] ======== Device 0 Memory Allocations:  ========
[13:33:52] Peak memory usage: 427MiB
[13:33:52] Number of allocations: 77042801
[13:33:52] ====

XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=<function log_loss at 0x000001D30175A3A0>, gamma=0,
                gpu_id=0, grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.001, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100000, n_jobs=0,
                num_parallel_tree=100000, objective='binary:logistic',
                predictor='auto', random_state=23, reg_alpha=0,
                sampling_method='uniform', ...)

In [7]:
p_train = model.predict(X_train)
p_test = model.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

acc_train = accuracy_score(p_train,y_train)
acc_test = accuracy_score(p_test,y_test)

rep_train = classification_report(y_train,p_train)
rep_test = classification_report(y_test,p_test)
model.save_model("model.json")
print(rep_train)
print(rep_test)


c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\xgboost\sklearn.py:722: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


              precision    recall  f1-score   support

           0       0.89      0.05      0.09     44559
           1       0.69      0.99      0.81    189479
           2       0.89      0.03      0.06     44666

   micro avg       0.69      0.69      0.69    278704
   macro avg       0.82      0.36      0.32    278704
weighted avg       0.75      0.69      0.58    278704
 samples avg       0.68      0.69      0.69    278704

              precision    recall  f1-score   support

           0       0.07      0.01      0.01      6886
           1       0.69      0.91      0.78     33141
           2       0.27      0.00      0.01      9157

   micro avg       0.68      0.62      0.65     49184
   macro avg       0.34      0.31      0.27     49184
weighted avg       0.52      0.62      0.53     49184
 samples avg       0.61      0.62      0.61     49184



In [ ]:
from xgboost import plot_importance ,plot_tree
import matplotlib as mpl

mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (200, 100)

plot_tree(model)
